In [2]:
import string
import scrapy
from scrapy import Request

## Website scrapping : (Extract)

In [4]:
class MangaDbSpider(scrapy.Spider):
    name = 'Manga'
    start_urls = ["https://myanimelist.net/anime.php",]

    def parse(self, response):
        xp = "//div[@id='horiznav_nav']//li/a/@href"
        return (Request(url, callback=self.parse_manga_list_page) for url in response.xpath(xp).extract())

    def parse_manga_list_page(self, response):
        for tr_sel in response.css('div.js-categories-seasonal tr ~ tr'):
            yield {
                "title":  tr_sel.css('a[id] strong::text').extract_first().strip(),
                "synopsis": tr_sel.css("div.pt4::text").extract_first(),
                "type_": tr_sel.css('td:nth-child(3)::text').extract_first().strip(),
                "episodes": tr_sel.css('td:nth-child(4)::text').extract_first().strip(), 
                "rating": tr_sel.css('td:nth-child(5)::text').extract_first().strip()
            }
            
        next_urls = response.xpath("//div[@class='spaceit']//a/@href").extract()
        for next_url in next_urls:
            yield Request(response.urljoin(next_url), callback=self.parse_anime_list_page)

## Loading in Database : (Load)

In [3]:
import pandas as pd
import json
import csv
import pymongo

In [7]:
#Importing the JSON file in DB

client = pymongo.MongoClient()
database = client['MangaDB']
collection = database['test']

In [8]:
df = pd.read_json("test.json")
df.head()

,title,synopsis,type_,episodes,rating
0,-50kg no Cinderella,"Hime, a chubby girl, is a university student w...",Manga,-,-
1,-6mm no Taboo,1. Half Moon 2. Private Gallery 3. Lover's Ret...,Manga,1,6.45
2,-Crow-,None,Manga,1,-
3,G,G takes place at these points we call boundari...,Manga,1,6.69
4,C Court,None,Manga,1,-


In [9]:
df = df.dropna()
df.head()

,title,synopsis,type_,episodes,rating
0,-50kg no Cinderella,"Hime, a chubby girl, is a university student w...",Manga,-,-
1,-6mm no Taboo,1. Half Moon 2. Private Gallery 3. Lover's Ret...,Manga,1,6.45
3,G,G takes place at these points we call boundari...,Manga,1,6.69
5,-Hitogatana-,In a world where it is possible to transfer hu...,Manga,-,7.27
7,-rain-,"""Because I love you."" On a rainy day, when Mai...",One-shot,-,5.48


In [10]:
df.rename(columns={'ID':'_id'}, inplace=True)
l_d =df.to_dict(orient='records')

In [12]:
def clean_record(record):
    return {k:v for k,v in record.items() if v}

l_d = [clean_record(record) for record in l_d]

In [13]:
collection.insert_many(l_d)

### Import by pipelines

In [ ]:
class MongoPipeline(object):

    collection_name = 'manga_items'

    def open_spider(self, spider):
        self.client = pymongo.MongoClient()
        self.db = self.client["MangaDB"]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.db[self.collection_name].insert_one(dict(item))
        return item

In [8]:
###TEST GET IMAGE URL
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome(executable_path='./chromedriver.exe')
driver.get('https://myanimelist.net/manga.php?letter=B')
results = []
content = driver.page_source
soup = BeautifulSoup(content)


def parse_image_urls(classes, location, source):
    for a in soup.findAll(attrs={'class': classes}):
        name = a.find(location)
        if name not in results:
            results.append(name.get(source))
            
parse_image_urls("hoverinfo_trigger", "img", "src")

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
#scrapy runspider mangaDBSpider.py
#scrapy crawl Manga -o mangaDB.json
#scrapy crawl Manga